<div class="alert alert-block alert-info">

## <center> GROUP PROJECT - TO GRANT OR NOT TO GRANT: DECIDING ON COMPENSATION BENEFITS </center> <br>
#  <center> <b> MLP Classifier </center> <br>
## <center> Fall Semester 2024-2025 <center>
<br>
<center> Group 46: <center> <br>
<center>Afonso Ascensão, 20240684 <br><center>
<center>Duarte Marques, 20240522 <br><center>
<center>Joana Esteves, 20240746 <br><center>
<center>Rita Serra, 20240515 <br><center>
<center>Rodrigo Luís, 20240742 <br><center>

<div>

**Description of contents:**
- Apply pipeline to preprocess the data.
- Implement MLP CLassifier algorithm, perform tuning of hyperparameters making use of gridsearch.
- Assessement of the model using cross validation.
- Generate prediction for the test sample.

**Table of Contents**
- [1. Import the needed Libraries](#importlibraries)
- [2. Import Dataset](#importdataset)
- [3. Preprocessing](#section_3)
- [4. MLP Classifier](#section_4)


<a class="anchor" id="section_1">

# 1. Import Libraries

</a>

In [1]:

import pandas as pd
import numpy as np

# Preprocessing
## Pipeline
from sklearn.pipeline import Pipeline
from joblib import load
from transformers import *
## Target Encoding
from sklearn.preprocessing import LabelEncoder

# Model algorithm 
from sklearn.neural_network import MLPClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, make_scorer, f1_score

# Cross validation, parameter tuning
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

# Resampling
from imblearn.over_sampling import SMOTE

# Define a seed
random_state = 42

# Display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)

<a class="anchor" id="section_2">

# 2. Import Dataset and Pipeline

</a>

In [2]:
# Train and validation w/ split, separate X and y to apply preprocessing
transformed_train_split = pd.read_parquet("transformed_train_split.parquet")
transformed_val_split = pd.read_parquet("transformed_val_split.parquet")

# Test set with predicted agreement column, apply preprocessing 
test_transformed_agreement = pd.read_parquet("test_transformed_agreement.parquet")

# Dataset with no split for cross validation, apply pipeline inside cross validation
transformed_train_data = pd.read_parquet("transformed_train_data.parquet")

In [3]:
# Load pipeline
pipeline = load('pipeline.joblib') 

<a class="anchor" id="section_3">

# 3. Preprocessing

</a>

In [4]:
# Separate X and y for train after split
X_train = transformed_train_split.drop(['Claim Injury Type'], axis = 1)
y_train = transformed_train_split['Claim Injury Type']

# Separate X and y for validation after split
X_val = transformed_val_split.drop(['Claim Injury Type'], axis = 1)
y_val = transformed_val_split['Claim Injury Type']

# Separate X and y for dataset before split
X = transformed_train_data.drop(['Claim Injury Type'], axis = 1)
y = transformed_train_data['Claim Injury Type']

In [5]:
# Apply encoding of y for train and validation sets

# Initialize target encoder
label_encoder = LabelEncoder()

# Encode target
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

In [6]:
# Apply preprocessing pipeline to the train, validation and test sets
X_train_preprocessed = pipeline.fit_transform(X_train, y_train_encoded)
X_val_preprocessed = pipeline.transform(X_val)
test_data_preprocessed = pipeline.transform(test_transformed_agreement)

In [11]:
print("Selected features:", X_train_preprocessed.columns.values)

Selected features: ['Attorney/Representative' 'Average Weekly Wage Log' 'C-2 Delivered'
 'Industry Code' 'Time Assembly to Hearing' 'Hearing Held'
 'Agreement Reached' 'C-3 Delivered on Time' 'Part of Body Group_Trunk'
 'Part of Body Group_Lower Extremities' 'IME-4 Count Log'
 'District Name_NYC' 'Part of Body Group_Upper Extremities' 'Gender'
 'Carrier Type_2A. SIF' 'Cause of Injury Group_X' 'Assembly Year'
 'Cause of Injury Group_VI']


<a class="anchor" id="section_4">

# 4. MLP Classifier

</a>

**Variables for model:**
- X_train_preprocessed;
- y_train_preprocessed;
- X_val_preprocessed;
- y_val_encoded;
- test_data_preprocessed.

**Variables for CV:**
- X: no preprocessing and no split;
- y: no preprocessing and no split;
- Apply pipeline inside cv.


In [27]:
''' 
# Starting point: 2 hidden layers, width of each hidden layer smaller than the width of the input layer (number of features)
# Deafult hyperparameters 

mlp = MLPClassifier(
    hidden_layer_sizes=(16,16),  
    random_state=random_state,            
)

mlp.fit(X_train_preprocessed, y_train_encoded)

# Predictions and Evaluation
y_pred_train = mlp.predict(X_train_preprocessed)
y_pred_val = mlp.predict(X_val_preprocessed)

print("\nClassification Report:\n", classification_report(y_train_encoded, y_pred_train))
print("\nClassification Report:\n", classification_report(y_val_encoded, y_pred_val))
mlp.fit(X_train_preprocessed, y_train_encoded)

# Predictions and Evaluation
y_pred_train = mlp.predict(X_train_preprocessed)
y_pred_val = mlp.predict(X_val_preprocessed)

print("\nClassification Report:\n", classification_report(y_train_encoded, y_pred_train))
print("\nClassification Report:\n", classification_report(y_val_encoded, y_pred_val)) '''

' \n# Starting point: 2 hidden layers, width of each hidden layer smaller than the width of the input layer (number of features)\n# Deafult hyperparameters \n\nmlp = MLPClassifier(\n    hidden_layer_sizes=(16,16),  \n    random_state=random_state,            \n)\n\nmlp.fit(X_train_preprocessed, y_train_encoded)\n\n# Predictions and Evaluation\ny_pred_train = mlp.predict(X_train_preprocessed)\ny_pred_val = mlp.predict(X_val_preprocessed)\n\nprint("\nClassification Report:\n", classification_report(y_train_encoded, y_pred_train))\nprint("\nClassification Report:\n", classification_report(y_val_encoded, y_pred_val))\nmlp.fit(X_train_preprocessed, y_train_encoded)\n\n# Predictions and Evaluation\ny_pred_train = mlp.predict(X_train_preprocessed)\ny_pred_val = mlp.predict(X_val_preprocessed)\n\nprint("\nClassification Report:\n", classification_report(y_train_encoded, y_pred_train))\nprint("\nClassification Report:\n", classification_report(y_val_encoded, y_pred_val)) '

In [28]:
''' # GridSearchCV

# Append the model to the imported pipeline
full_pipeline = Pipeline(
    pipeline.steps + [('model', MLPClassifier(max_iter=1000, random_state=random_state))]
)

# Parameter grid
param_grid = {
    'model__hidden_layer_sizes': [(64, 32), (64, 64)], # 2 layers with more neurons than starting point (16,16)
    'model__alpha': [0.001, 0.0001], # deafult 0.0001 vs. 0.001
    'model__learning_rate_init': [0.01, 0.001],  # deafult 0.001 vs. 0.01
}

# GridSearchCV
grid_search = GridSearchCV(
    estimator=full_pipeline,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1
)

# Encode target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Fit GridSearchCV
grid_search.fit(X, y_encoded)

# Display best parameters and best cv score 
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_) '''

' # GridSearchCV\n\n# Append the model to the imported pipeline\nfull_pipeline = Pipeline(\n    pipeline.steps + [(\'model\', MLPClassifier(max_iter=1000, random_state=random_state))]\n)\n\n# Parameter grid\nparam_grid = {\n    \'model__hidden_layer_sizes\': [(64, 32), (64, 64)], # 2 layers with more neurons than starting point (16,16)\n    \'model__alpha\': [0.001, 0.0001], # deafult 0.0001 vs. 0.001\n    \'model__learning_rate_init\': [0.01, 0.001],  # deafult 0.001 vs. 0.01\n}\n\n# GridSearchCV\ngrid_search = GridSearchCV(\n    estimator=full_pipeline,\n    param_grid=param_grid,\n    scoring=\'f1_macro\',\n    cv=3,\n    n_jobs=-1\n)\n\n# Encode target\nlabel_encoder = LabelEncoder()\ny_encoded = label_encoder.fit_transform(y)\n\n# Fit GridSearchCV\ngrid_search.fit(X, y_encoded)\n\n# Display best parameters and best cv score \nprint("Best parameters:", grid_search.best_params_)\nprint("Best score:", grid_search.best_score_) '

In [29]:
''' # Same number of neurons than best solution from gridsearch but with more layers

mlp = MLPClassifier(
    hidden_layer_sizes=(32,32,32,32),  
    activation='relu',              
    solver='adam',                  
    learning_rate_init=0.001,      
    max_iter=1000,                   
    alpha=0.0001,                
    random_state=random_state,                
)

mlp.fit(X_train_preprocessed, y_train_encoded)

# Predictions and Evaluation
y_pred_train = mlp.predict(X_train_preprocessed)
y_pred_val = mlp.predict(X_val_preprocessed)

print("\nClassification Report:\n", classification_report(y_train_encoded, y_pred_train))
print("\nClassification Report:\n", classification_report(y_val_encoded, y_pred_val)) '''


' # Same number of neurons than best solution from gridsearch but with more layers\n\nmlp = MLPClassifier(\n    hidden_layer_sizes=(32,32,32,32),  \n    activation=\'relu\',              \n    solver=\'adam\',                  \n    learning_rate_init=0.001,      \n    max_iter=1000,                   \n    alpha=0.0001,                \n    random_state=random_state,                \n)\n\nmlp.fit(X_train_preprocessed, y_train_encoded)\n\n# Predictions and Evaluation\ny_pred_train = mlp.predict(X_train_preprocessed)\ny_pred_val = mlp.predict(X_val_preprocessed)\n\nprint("\nClassification Report:\n", classification_report(y_train_encoded, y_pred_train))\nprint("\nClassification Report:\n", classification_report(y_val_encoded, y_pred_val)) '

In [30]:
''' # Around the same number of neurons than best solution from gridsearch but with more layers
mlp = MLPClassifier(
    hidden_layer_sizes=(43,43,43),  
    activation='relu',              
    solver='adam',                  
    learning_rate_init=0.001,      
    max_iter=1000,                   
    alpha=0.0001,                
    random_state=random_state,                
)

mlp.fit(X_train_preprocessed, y_train_encoded)

# Predictions and Evaluation
y_pred_train = mlp.predict(X_train_preprocessed)
y_pred_val = mlp.predict(X_val_preprocessed)

print("\nClassification Report:\n", classification_report(y_train_encoded, y_pred_train))
print("\nClassification Report:\n", classification_report(y_val_encoded, y_pred_val)) '''

' # Around the same number of neurons than best solution from gridsearch but with more layers\nmlp = MLPClassifier(\n    hidden_layer_sizes=(43,43,43),  \n    activation=\'relu\',              \n    solver=\'adam\',                  \n    learning_rate_init=0.001,      \n    max_iter=1000,                   \n    alpha=0.0001,                \n    random_state=random_state,                \n)\n\nmlp.fit(X_train_preprocessed, y_train_encoded)\n\n# Predictions and Evaluation\ny_pred_train = mlp.predict(X_train_preprocessed)\ny_pred_val = mlp.predict(X_val_preprocessed)\n\nprint("\nClassification Report:\n", classification_report(y_train_encoded, y_pred_train))\nprint("\nClassification Report:\n", classification_report(y_val_encoded, y_pred_val)) '

- The model with 2 hidden layers has a better f1 macro score than 3 and 4 hidden layers, and gets predictions for one more class.

**Model with selected hyperparameters:**

In [20]:
mlp = MLPClassifier(
    hidden_layer_sizes=(64,64),    # 2 hidden layers W/ 64 neurons each
    activation='relu',             # ReLU activation function - default
    solver='adam',                 # Adam solver - deafult
    learning_rate_init=0.001,      # Initial learning rate for Adam 
    max_iter=1000,                 # Maximum number of iterations
    alpha=0.0001,                 # L2 regularization to prevent overfitting 
    random_state=random_state,     
)

mlp.fit(X_train_preprocessed, y_train_encoded)

# Predictions and Evaluation
y_pred_train = mlp.predict(X_train_preprocessed)
y_pred_val = mlp.predict(X_val_preprocessed)

print("\nClassification Report:\n", classification_report(y_train_encoded, y_pred_train))
print("\nClassification Report:\n", classification_report(y_val_encoded, y_pred_val))

/Users/joanaesteves/.ml_project/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.38      0.49     11229
           1       0.83      0.98      0.90    261970
           2       0.48      0.09      0.15     62016
           3       0.76      0.81      0.78    133656
           4       0.63      0.66      0.64     43452
           5       0.45      0.01      0.01      3790
           6       0.00      0.00      0.00        87
           7       0.55      0.24      0.34       423

    accuracy                           0.78    516623
   macro avg       0.55      0.40      0.41    516623
weighted avg       0.74      0.78      0.74    516623


Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.39      0.50      1248
           1       0.83      0.98      0.89     29108
           2       0.43      0.08      0.13      6890
           3       0.75      0.80      0.78     14851
           4       0.62     

/Users/joanaesteves/.ml_project/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/joanaesteves/.ml_project/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/joanaesteves/.ml_project/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

- The scores are similar for train and validation.
- The model is not predicting class 6, most likely because it does not have enough observations.
- The minority classes have a very low f1 score.
- The f1 macro scores are low for both train and validation which indicate the model may be underfitting.

**Cross-validation w/ 5 splits for final assessement of the model with selected hyperparameters:**

In [25]:
# Append the model to the imported pipeline
full_pipeline = Pipeline(
    pipeline.steps + [('model', mlp)]
)

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

# Encode target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Use F1-macro because of class imbalance
scorer = make_scorer(f1_score, average='macro')

# Get scores for cross-validation
# Apply preprocessing inside cv for X and y_encoded
cv_scores = cross_val_score(full_pipeline, X, y_encoded, cv=cv, scoring=scorer)

# Print results
print("Cross-validation scores (F1-macro):", cv_scores)
print("Mean CV score:", cv_scores.mean())

Cross-validation scores (F1-macro): [0.39539424 0.40278262 0.38386223 0.41036415 0.38424573]
Mean CV score: 0.3953297931932184


- Mean CV f1 macro: 0.3953297931932184
- This value will be compared with the other models mean cv scores to select the best model.

**Predictions for test set using all incial train data:**

In [ ]:
# Encode target
y_encoded = label_encoder.fit_transform(y)

# Fit pipeline on X and apply tranformations
X_preprocessed = pipeline.fit_transform(X, y_encoded)

# Apply pipeline fitted on X
test_data_preprocessed_X = pipeline.transform(test_transformed_agreement)

# Fit on X 
mlp.fit(X_preprocessed,y_encoded)

# Make predictions on test data
y_pred = mlp.predict(test_data_preprocessed_X)

# Get original y for submission
y_pred_categorical = label_encoder.inverse_transform(y_pred) 

submission = pd.DataFrame({
    "Claim Identifier": test_data_preprocessed.index,
    "Claim Injury Type": y_pred_categorical
})


# Save to CSV to upload on kaggle
submission.to_csv("Group46_versionX.csv", index=False)

- Kaggle f1 macro score from test: 0.35491.